# FedCAD: Centralized vs Federated (orchestration)

This notebook launches centralized and federated experiments via existing entry points:
- Centralized: `python train_central.py`
- Federated: `flwr run .`

Metrics tracked in W&B:
- Centralized: `train_loss`, `test_loss`, `test_acc` (already logged by `train_central.py`)
- Federated: depends on your Flower app (ensure it logs to the same W&B project/group)

Recommended seeds for a seminar: 3 (e.g., `[0, 1, 2]`). Use 5 if time permits for tighter confidence intervals.

In [77]:
import os, sys, subprocess, shutil, time
import torch

# W&B configuration
def load_wandb_config():
    """Load WandB credentials from config file (optional for reproducibility)"""
    config = {}
    config_path = './.wandb_config'
    try:
        with open(config_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    config[key] = value
        return config.get('WANDB_ENTITY'), config.get('WANDB_PROJECT')
    except FileNotFoundError:
        return None, None

entity, project_name = load_wandb_config()
WANDB_MODE = "online"               # set to "offline" or "disabled" if needed

SEEDS = list(range(0, 66)) 

# Centralized hyperparameters
EPOCHS_CENTRAL = 30
LR_CENTRAL = 1e-3
EVAL_INTERVAL = 1

# Device
USE_GPU_FLAG = torch.cuda.is_available()
print("Device:", "cuda" if USE_GPU_FLAG else "cpu")
print("Seeds:", SEEDS)

def run_cmd(cmd_list, extra_env=None, cwd=None):
    env = os.environ.copy()
    if extra_env:
        env.update({k: str(v) for k, v in extra_env.items()})
    print("\n>>>", " ".join(cmd_list))
    print("    in:", cwd or os.getcwd())
    print("    with env overrides:", extra_env or {})
    proc = subprocess.Popen(cmd_list, cwd=cwd, env=env)
    proc.wait()
    if proc.returncode != 0:
        raise RuntimeError(f"Command failed with exit code {proc.returncode}: {' '.join(cmd_list)}")

Device: cpu
Seeds: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]


## Centralized experiments

Runs `train_central.py` once per seed. The script already logs: `train_loss`, `test_loss`, `test_acc`.

Note: `train_central.py` does not expose a `--seed` flag. If you need strict reproducibility per seed, add seeding to the script; otherwise each run will use random initialization.

In [78]:
for seed in SEEDS:
    # W&B grouping so runs are easy to compare
    env_overrides = {
        "WANDB_PROJECT": project_name,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "centralized",
        "WANDB_TAGS": "centralized,notebook",
        "WANDB_ENTITY": entity,

        # Optional: many apps read SEED from env; train_central.py currently does not
        "SEED": seed,
    }

    cmd = [sys.executable, "train_central.py",
           "--epochs", str(EPOCHS_CENTRAL),
           "--lr", str(LR_CENTRAL),
           "--eval-interval", str(EVAL_INTERVAL),
           "--wandb-mode", WANDB_MODE]
    if USE_GPU_FLAG:
        cmd.append("--gpu")

    print(f"\n=== Centralized run, seed={seed} ===")
    run_cmd(cmd, extra_env=env_overrides, cwd=os.getcwd())
    time.sleep(1)  # small gap between runs


=== Centralized run, seed=0 ===

>>> /Users/zoe/Desktop/Fed-CAD/.venv/bin/python train_central.py --epochs 30 --lr 0.001 --eval-interval 1 --wandb-mode online
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'centralized', 'WANDB_TAGS': 'centralized,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 0}


wandb: Currently logged in as: zoematr (louisewiljander-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: setting up run 7cowldos
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/zoe/Desktop/Fed-CAD/wandb/run-20260224_120950-7cowldos
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run central_lr0.001_ep30
wandb: ⭐️ View project at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD
wandb: 🚀 View run at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD/runs/7cowldos


Epoch 1/30 | train_loss=0.596 | test_loss=0.497 | test_acc=0.764 | time=0.0min
Epoch 2/30 | train_loss=0.544 | test_loss=0.461 | test_acc=0.782 | time=0.0min
Epoch 3/30 | train_loss=0.513 | test_loss=0.444 | test_acc=0.791 | time=0.0min
Epoch 4/30 | train_loss=0.520 | test_loss=0.470 | test_acc=0.736 | time=0.0min
Epoch 5/30 | train_loss=0.477 | test_loss=0.411 | test_acc=0.791 | time=0.0min
Epoch 6/30 | train_loss=0.462 | test_loss=0.423 | test_acc=0.782 | time=0.0min
Epoch 7/30 | train_loss=0.449 | test_loss=0.489 | test_acc=0.727 | time=0.0min
Epoch 8/30 | train_loss=0.475 | test_loss=0.432 | test_acc=0.782 | time=0.0min
Epoch 9/30 | train_loss=0.434 | test_loss=0.384 | test_acc=0.845 | time=0.0min
Epoch 10/30 | train_loss=0.423 | test_loss=0.381 | test_acc=0.827 | time=0.0min
Epoch 11/30 | train_loss=0.403 | test_loss=0.384 | test_acc=0.818 | time=0.0min
Epoch 12/30 | train_loss=0.388 | test_loss=0.395 | test_acc=0.809 | time=0.0min
Epoch 13/30 | train_loss=0.369 | test_loss=0.483 

wandb: updating run metadata
wandb: uploading history steps 0-29, summary, console lines 0-33
wandb: 
wandb: Run history:
wandb: elapsed_time_min ▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
wandb:            epoch ▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
wandb:         test_acc ▃▄▅▂▅▄▁▄█▇▆▆▂▃▄▂▃▇▇▅▇█▇█▆▄▇█▇▇
wandb:        test_loss █▆▅▆▄▄▇▅▂▂▂▃▇▄▅█▆▁▁▃▁▂▂▂▂▇▂▃▃▄
wandb:       train_loss █▇▇▇▆▆▆▆▆▅▅▅▄▅▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁
wandb: 
wandb: Run summary:
wandb: elapsed_time_min 0.08876
wandb:            epoch 30
wandb:    final_auc_roc 0.85165
wandb:         final_f1 0.82148
wandb:  final_precision 0.82718
wandb:     final_recall 0.83636
wandb:   final_test_acc 0.83636
wandb:  final_test_loss 0.41423
wandb: final_train_loss 0.14242
wandb:         test_acc 0.83636
wandb:               +4 ...
wandb: 
wandb: 🚀 View run central_lr0.001_ep30 at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD/runs/7cowldos
wandb: ⭐️ View project at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-o

Results saved to results.csv

=== Centralized run, seed=1 ===

>>> /Users/zoe/Desktop/Fed-CAD/.venv/bin/python train_central.py --epochs 30 --lr 0.001 --eval-interval 1 --wandb-mode online
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'centralized', 'WANDB_TAGS': 'centralized,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 1}


...

Results saved to results.csv

=== Centralized run, seed=65 ===

>>> /Users/zoe/Desktop/Fed-CAD/.venv/bin/python train_central.py --epochs 30 --lr 0.001 --eval-interval 1 --wandb-mode online
    in: /Users/zoe/Desktop/Fed-CAD
    with env overrides: {'WANDB_PROJECT': 'FedCAD', 'WANDB_MODE': 'online', 'WANDB_RUN_GROUP': 'centralized', 'WANDB_TAGS': 'centralized,notebook', 'WANDB_ENTITY': 'louisewiljander-ludwig-maximilianuniversity-of-munich', 'SEED': 65}


wandb: Currently logged in as: zoematr (louisewiljander-ludwig-maximilianuniversity-of-munich) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: setting up run sxi2mxga
wandb: Tracking run with wandb version 0.23.0
wandb: Run data is saved locally in /Users/zoe/Desktop/Fed-CAD/wandb/run-20260224_122558-sxi2mxga
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run central_lr0.001_ep30
wandb: ⭐️ View project at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD
wandb: 🚀 View run at https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD/runs/sxi2mxga


Epoch 1/30 | train_loss=0.601 | test_loss=0.494 | test_acc=0.764 | time=0.0min
Epoch 2/30 | train_loss=0.560 | test_loss=0.506 | test_acc=0.764 | time=0.0min
Epoch 3/30 | train_loss=0.534 | test_loss=0.457 | test_acc=0.764 | time=0.0min
Epoch 4/30 | train_loss=0.507 | test_loss=0.448 | test_acc=0.764 | time=0.0min
Epoch 5/30 | train_loss=0.497 | test_loss=0.436 | test_acc=0.764 | time=0.0min
Epoch 6/30 | train_loss=0.477 | test_loss=0.415 | test_acc=0.764 | time=0.0min
Epoch 7/30 | train_loss=0.464 | test_loss=0.394 | test_acc=0.764 | time=0.0min
Epoch 8/30 | train_loss=0.444 | test_loss=0.431 | test_acc=0.764 | time=0.0min
Epoch 9/30 | train_loss=0.458 | test_loss=0.426 | test_acc=0.764 | time=0.0min
Epoch 10/30 | train_loss=0.426 | test_loss=0.416 | test_acc=0.764 | time=0.0min
Epoch 11/30 | train_loss=0.424 | test_loss=0.398 | test_acc=0.764 | time=0.0min
Epoch 12/30 | train_loss=0.399 | test_loss=0.377 | test_acc=0.764 | time=0.0min
Epoch 13/30 | train_loss=0.382 | test_loss=0.376 

wandb: updating run metadata
wandb: uploading history steps 0-29, summary, console lines 0-33
wandb: 
wandb: Run history:
wandb: elapsed_time_min ▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
wandb:            epoch ▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
wandb:         test_acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▆▇█▇▇▆▆▆██▇▆▇▆█
wandb:        test_loss ▇█▅▅▄▃▂▄▄▃▂▁▁▃▅▂▁▁▂▁▂▂▃▂▃▃▄▄▅▄
wandb:       train_loss █▇▇▆▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▂▂▂▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb: elapsed_time_min 0.09194
wandb:            epoch 30
wandb:    final_auc_roc 0.84066
wandb:         final_f1 0.87818
wandb:  final_precision 0.90455
wandb:     final_recall 0.89091
wandb:   final_test_acc 0.89091
wandb:  final_test_loss 0.43534
wandb: final_train_loss 0.23131
wandb:         test_acc 0.89091
wandb:               +4 ...
wandb: 
wandb: 🚀 View run central_lr0.001_ep30 at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-of-munich/FedCAD/runs/sxi2mxga
wandb: ⭐️ View project at: https://wandb.ai/louisewiljander-ludwig-maximilianuniversity-o

Results saved to results.csv


In [ ]:
import os

# Check current state
print("1. Current directory:", os.getcwd())
print("2. pyproject.toml exists:", os.path.exists("pyproject.toml"))
print("3. Full path:", os.path.abspath("pyproject.toml"))

# List files to confirm
print("\n4. Files in current directory:")
for f in os.listdir("."):
    print(f"   {f}")

## Federated experiments (Flower Simulation Engine)

Runs `flwr run .` once per seed. Ensure your Flower app logs to W&B (same project) with metrics comparable to centralized (e.g., `global_train_loss`, `global_test_loss`, `global_test_acc`).

Tip: If your Flower app reads `WANDB_*` env vars and `SEED`, the grouping and seeds below will apply.

In [ ]:
def run_flower(extra_env=None):
    # Prefer CLI if available; fallback to `python -m flwr`
    if shutil.which("flwr"):
        run_cmd(["flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())
    else:
        run_cmd([sys.executable, "-m", "flwr", "run", "."], extra_env=extra_env, cwd=os.getcwd())

entity, project_name = load_wandb_config()

for seed in SEEDS:
    env_overrides = {
        "WANDB_PROJECT": project_name,
        "WANDB_MODE": WANDB_MODE,
        "WANDB_RUN_GROUP": "federated",
        "WANDB_TAGS": "federated,notebook",
        "WANDB_ENTITY": entity,
        "SEED": seed,
    }
    print(f"\n=== Federated run (Flower), seed={seed} ===")
    run_flower(extra_env=env_overrides)
    time.sleep(1)